In [1]:
# Name Dipika Sharma
# DSC 530 T301 Data Exploration
# Week 9

# Assignment 9.2 - (11.1, 11.3, 11.4)

http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT

In [39]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/first.py")

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dct")
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dat.gz"
)
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dct")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dat.gz")

Downloaded 2002FemResp.dct
Downloaded 2002FemResp.dat.gz


In [40]:
import numpy as np
import pandas as pd

import thinkstats2
import thinkplot

In [41]:
import first

live, firsts, others = first.MakeFrames()

# Exercise 11.1

Exercise 11.1 Suppose one of your co-workers is expecting a baby and you
are participating in an office pool to predict the date of birth. Assuming that
bets are placed during the 30th week of pregnancy, what variables could you
use to make the best prediction? You should limit yourself to variables that
are known before the birth, and likely to be available to the people in the
pool.

In [11]:
# bets are placed during the 30th week of pregnancy

live_30 = live[live.prglngth>30]

In [12]:
live_30.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,laborfor_i,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw,totalwgt_lb
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,8.8125
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,7.8750
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,9.1250
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,7.0000
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,6.1875


In [26]:
# Lets see pregnancy length as a function of few variables that I think have significant effect on pregnancy length
# Using age, birthorder, race and nbralive for creating a model

import statsmodels.formula.api as smf

formula = 'prglngth ~ agepreg>30 + birthord==1 + race==2 + nbrnaliv>1'
model = smf.ols(formula, data=live_30)
results = model.fit()

In [30]:
# lets see the summary of the model.

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     25.71
Date:                Wed, 11 May 2022   Prob (F-statistic):           3.26e-21
Time:                        16:47:27   Log-Likelihood:                -18247.
No. Observations:                8884   AIC:                         3.650e+04
Df Residuals:                    8879   BIC:                         3.654e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.7625      0.040    970.738      0.000      38.684      38.841
agepreg > 30[T.True]     -0.0039      0.051     -0.076      0.940      -0.105       0.097
birthord == 1[T.True]     0.1009      0.041      2.467      0.014       0.021       0.181
race == 2[T.True]         0.1394      0.042      3.294      0.001       0.056       0.222
nbrnaliv > 1[T.True]     -1.4942      0.165     -9.082      0.000      -1.817      -1.172
==============================================================================
Omnibus:                     1587.472   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6160.616
Skew:                          -0.852   Prob(JB):                         0.00
Kurtosis:                       6.707   Cond. No.                         11.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

As we can see the R-squared value is between 0 to 1 so the created mmodel is a fit.
The std err varialble is very low which means the selected model is accurate.
If we exclude the age then for remaining variable p-value is less than 0.05 which is considered to be statistically significant

# Exercise 11.3

Exercise 11.3 If the quantity you want to predict is a count, you can use
Poisson regression, which is implemented in StatsModels with a function
called poisson. It works the same way as ols and logit. As an exercise,
let’s use it to predict how many children a woman has born; in the NSFG
dataset, this variable is called numbabes.
Suppose you meet a woman who is 35 years old, black, and a college graduate
whose annual household income exceeds $75,000. How many children would
you predict she has born?


In [87]:
import nsfg
live = live[live.prglngth>30] # selecting records which has longer than 30 weeks pregnancies.
resp = nsfg.ReadFemResp()  # Reading respondent file
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r') # Joining the live and resp table using caseid.
join.shape

(8884, 3331)

In [88]:
#for col in join.columns:
#    print(col)

In [93]:
# Lets use the nonlinear model of age.

join.numbabes.replace([97], np.nan, inplace=True)
join['age2'] = join.age_r**2
#join['age3'] = join.age_r**3

In [97]:
# Lets create a model using variables age, age squared, age cube, race, toincr and educat

formula = 'numbabes ~ age_r + age2 + C(race) + totincr + educat'

model = smf.poisson(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.677002
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8877
Method:                           MLE   Df Model:                            6
Date:                Sat, 14 May 2022   Pseudo R-squ.:                 0.03686
Time:                        16:49:19   Log-Likelihood:                -14898.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                3.681e-243
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.0324      0.169     -6.098      0.000      -1.364      -0.701
C(race)[T.2]    -0.1401      0.015     -9.479      0.000      -0.169      -0.111
C(race)[T.3]    -0.0991      0.025     -4.029      0.000      -0.147      -0.051
age_r            0.1556      0.010     15.006      0.000       0.135       0.176
age2            -0.0020      0.000    -13.102      0.000      -0.002      -0.002
totincr         -0.0187      0.002     -9.830      0.000      -0.022      -0.015
educat          -0.0471      0.003    -16.076      0.000      -0.053      -0.041
================================================================================
"""

Lets now predict the number of children for a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000.

In [105]:
# Since the income is greater then $75,000 and number 1 stands for 5000$, and the range is 1-14.
# we using 14 as total income as that is the max in available range as mentioned in the book.

# Book also says race variable is define as 1 for black, 2 for white, and 3 for other

columns = ['age_r', 'age2', 'age3', 'race', 'totincr', 'educat']
new = pd.DataFrame([[35, 35**2, 35**3, 1, 14, 16]], columns=columns)
results.predict(new)

0    2.496802
dtype: float64

As we can see for created model the women who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000 has 2 kids.

# Exercise 11.4

Exercise 11.4 If the quantity you want to predict is categorical, you can use
multinomial logistic regression, which is implemented in StatsModels with a
function called mnlogit. As an exercise, let’s use it to guess whether a woman
is married, cohabitating, widowed, divorced, separated, or never married; in
the NSFG dataset, marital status is encoded in a variable called rmarital.
Suppose you meet a woman who is 25 years old, white, and a high school
graduate whose annual household income is about $45,000. What is the
probability that she is married, cohabitating, etc?

In [106]:
# Lets create a model usng all important variable.

formula='rmarital ~ age_r + age2 + C(race) + totincr + educat'
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.084053
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8849
Method:                           MLE   Df Model:                           30
Date:                Sat, 14 May 2022   Pseudo R-squ.:                  0.1682
Time:                        16:59:39   Log-Likelihood:                -9630.7
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        9.0156      0.805     11.199      0.000       7.438      10.593
C(race)[T.2]    -0.9237      0.089    -10.418      0.000      -1.097      -0.750
C(race)[T.3]    -0.6179      0.136     -4.536      0.000      -0.885      -0.351
age_r           -0.3635      0.051     -7.150      0.000      -0.463      -0.264
age2             0.0048      0.001      6.103      0.000       0.003       0.006
totincr         -0.1310      0.012    -11.337      0.000      -0.154      -0.108
educat          -0.1953      0.019    -10.424      0.000      -0.232      -0.159
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.9570      3.020      0.979      0.328      -2.963       8.877
C(race)[T.2]    -0.4411      0.237     -1.863      0.062      -0.905       0.023
C(race)[T.3]     0.0591      0.336      0.176      0.860      -0.600       0.718
age_r           -0.3177      0.177     -1.798      0.072      -0.664       0.029
age2             0.0064      0.003      2.528      0.011       0.001       0.011
totincr         -0.3258      0.032    -10.175      0.000      -0.389      -0.263
educat          -0.0991      0.048     -2.050      0.040      -0.194      -0.004
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -3.5238      1.205     -2.924      0.003      -5.886      -1.162
C(race)[T.2]    -0.3213      0.093     -3.445      0.001      -0.504      -0.139
C(race)[T.3]    -0.7706      0.171     -4.509      0.000      -1.106      -0.436
age_r            0.1155      0.071      1.626      0.104      -0.024       0.255
age2            -0.0007      0.001     -0.701      0.483      -0.003       0.001
totincr         -0.2276      0.012    -19.621      0.000      -0.250      -0.205
educat           0.0667      0.017      3.995      0.000       0.034       0.099
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.8963      1.305     -2.220      0.026      -5.453      -0.339
C(race)[T.2]    -1.0407      0.104    -10.038      0.000      -1.244      -0.837
C(race)[T.3]    -0.5661      0.156     -3.635      0.000      -0.871      -0.261
age_r            0.2411      0.079      3.038      0.002       0.086       0.397
age2            -0.0035      0.001     -2.977      0.003      -0.006      -0.001
totincr         -0.2932      0.015    -20.159      0.000      -0.322      -0.265
educat          -0.0174      0.021     -0.813      0.416      -0.059       0.025
-

Lets make a prediction for a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000.

In [109]:
# women is white so race will be 2 as defined in book.
# salary is about 45000$ so dividing it by 5000$ we will get 9

columns = ['age_r', 'age2', 'race', 'totincr', 'educat']
new = pd.DataFrame([[25, 25**2, 2, 9, 12]], columns=columns)
results.predict(new)

,0,1,2,3,4,5
0,0.664022,0.145425,0.002656,0.046617,0.034194,0.107085


Looking at the above result we can say 66% chances that the women is married, 14.5% chances that women is not married but living with someone and then so on.

In [108]:
# If the women has household income 55000$ then the prediction will be

columns = ['age_r', 'age2', 'race', 'totincr', 'educat']
new = pd.DataFrame([[25, 25**2, 2, 11, 12]], columns=columns)
results.predict(new)

,0,1,2,3,4,5
0,0.750028,0.126397,0.001564,0.033403,0.021485,0.067122


That means if white high school graduate women who is 25 year old and has household income of 55000$.
Then there is 75% chances that the women is married and 12.6% chances that the women is living with someone and not married. And so on.